In [ ]:
import torch as tc
from typing import Union
import pathlib

class Loader(ABC):
    def load_case(self, input_path: Union[str, pathlib.Path], loading_params: dict) -> tc.Tensor:
        with open(input_path, 'r') as file:
            lines = file.readlines()
        data = []
        for line in lines:
            line_data = line.strip().split(',')
            line_data = [float(value) for value in line_data]
            data.append(line_data)
        tensor = tc.tensor(data)

        return tensor


In [ ]:
import torch as tc
from typing import Union
import pathlib
import librosa

class RawLoader(Loader):
    def __init__(self, sample_rate: int = 44100):
        
        self.sample_rate = sample_rate

    def load_case(self, input_path: Union[str, pathlib.Path], loading_params: dict) -> tc.Tensor:
       
        waveform, _ = librosa.load(input_path, sr=self.sample_rate, mono=False)  
        if waveform.shape[0] == 2:
            tensor = tc.from_numpy(waveform)
        elif waveform.shape[0] == 1:
            tensor = tc.from_numpy(waveform).repeat(2, 1)
        else:
            raise ValueError("Nieobsługiwana liczba kanałów w pliku dźwiękowym.")

        tensor = tensor.unsqueeze(0)  

        return tensor


In [ ]:
import torch as tc
from typing import Union
import pathlib
import librosa

class SpectrogramLoader(Loader):
    def __init__(self, sample_rate: int = 44100, n_fft: int = 2048, hop_length: int = 512, stereo: bool = False): #tutaj nie wiem na ile Hz chcemy przyjąć więc ustawiłam domyślnie tyle 
       
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.stereo = stereo

    def load_case(self, input_path: Union[str, pathlib.Path], loading_params: dict) -> tc.Tensor:
        
        waveform, _ = librosa.load(input_path, sr=self.sample_rate, mono=not self.stereo)

        spectrogram = tc.stft(tc.from_numpy(waveform), n_fft=self.n_fft, hop_length=self.hop_length, center=True)

        return tc.abs(spectrogram)  

In [ ]:
class MelspectrogramLoader(Loader):
    def __init__(self, sample_rate: int = 44100, n_fft: int = 2048, hop_length: int = 512, n_mels: int = 128, stereo: bool = False):

        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.stereo = stereo

    def load_case(self, input_path: Union[str, pathlib.Path], loading_params: dict) -> tc.Tensor:

        waveform, _ = librosa.load(input_path, sr=self.sample_rate, mono=not self.stereo)

        mel_spec = librosa.feature.melspectrogram(waveform, sr=self.sample_rate, n_fft=self.n_fft, hop_length=self.hop_length, n_mels=self.n_mels)

        return tc.from_numpy(mel_spec)
